In [42]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

## DNN

In [32]:
data_dnn = pd.read_json('results_dnn.json').T

#data_dnn

In [36]:
metrics_dnn = []

for i, row in data_dnn.iterrows():

    accuracy_fold_train = []
    accuracy_fold_test = []

    recall_fold_train = []
    recall_fold_test = []

    precision_fold_train = []
    precision_fold_test = []

    for iteration in row["results"]:
        
        loss = iteration["loss_values"]
        conf_mat_train = np.array(iteration["confusion_matrix_train"])
        conf_mat_test = np.array(iteration["confusion_matrix_val"])
        
        # Compute accuracy (7 classes)
        total_train = conf_mat_train.sum()
        total_test = conf_mat_test.sum()

        accuracy_train = conf_mat_train.diagonal().sum() / total_train if total_train > 0 else 0
        accuracy_test = conf_mat_test.diagonal().sum() / total_test if total_test > 0 else 0

        # Compute recall and precision (per class)
        recall_train = np.array([
            conf_mat_train[i, i] / conf_mat_train[i, :].sum() if conf_mat_train[i, :].sum() > 0 else 0
            for i in range(7)
        ])
        recall_test = np.array([
            conf_mat_test[i, i] / conf_mat_test[i, :].sum() if conf_mat_test[i, :].sum() > 0 else 0
            for i in range(7)
        ])

        precision_train = np.array([
            conf_mat_train[i, i] / conf_mat_train[:, i].sum() if conf_mat_train[:, i].sum() > 0 else 0
            for i in range(7)
        ])
        precision_test = np.array([
            conf_mat_test[i, i] / conf_mat_test[:, i].sum() if conf_mat_test[:, i].sum() > 0 else 0
            for i in range(7)
        ])

        accuracy_fold_train.append(accuracy_train)
        accuracy_fold_test.append(accuracy_test)

        recall_fold_train.append(recall_train)
        recall_fold_test.append(recall_test)

        precision_fold_train.append(precision_train)
        precision_fold_test.append(precision_test)

    # Compute mean across folds
    accuracy_train = np.mean(accuracy_fold_train)
    accuracy_test = np.mean(accuracy_fold_test)

    recall_train = np.mean(recall_fold_train, axis=0)
    recall_test = np.mean(recall_fold_test, axis=0)

    precision_train = np.mean(precision_fold_train, axis=0)
    precision_test = np.mean(precision_fold_test, axis=0)

    average_recall_train = np.mean(recall_train)
    average_recall_test = np.mean(recall_test)

    average_precision_train = np.mean(precision_train)
    average_precision_test = np.mean(precision_test)

    f1_train_average = 2 * (average_precision_train * average_recall_train) / (average_precision_train + average_recall_train)
    f1_test_average = 2 * (average_precision_test * average_recall_test) / (average_precision_test + average_recall_test) 

    row = row.drop("results")
    row_dict = row.to_dict()

    metrics_dnn.append({
        **row_dict,
        "iteration": i,
        "accuracy_train": accuracy_train,
        "accuracy_test": accuracy_test,
        "recall_train": recall_train.tolist(),  # Convert to list for DataFrame compatibility
        "recall_test": recall_test.tolist(),
        "averaged_recall_train": average_recall_train,
        "averaged_recall_test": average_recall_test,
        "precision_train": precision_train.tolist(),
        "precision_test": precision_test.tolist(),
        "averaged_precision_train": average_precision_train,
        "averaged_precision_test": average_precision_test,
        "f1_train_average": f1_train_average,
        "f1_test_average": f1_test_average
    })

metrics_dnn = pd.DataFrame(metrics_dnn)
#metrics_dnn


## CNN

In [27]:
def get_metrics(data : pd.DataFrame):

    metrics_cnn = []

    for i, row in data.iterrows():

        accuracy_fold_train = []
        accuracy_fold_test = []

        recall_fold_train = []
        recall_fold_test = []

        precision_fold_train = []
        precision_fold_test = []

        for iteration in row["results"]:
            
            loss = iteration["loss_values"]
            conf_mat_train = np.array(iteration["confusion_matrix_train"])
            conf_mat_test = np.array(iteration["confusion_matrix_val"])
            
            # Compute accuracy (7 classes)
            total_train = conf_mat_train.sum()
            total_test = conf_mat_test.sum()

            accuracy_train = conf_mat_train.diagonal().sum() / total_train if total_train > 0 else 0
            accuracy_test = conf_mat_test.diagonal().sum() / total_test if total_test > 0 else 0

            # Compute recall and precision (per class)
            recall_train = np.array([
                conf_mat_train[i, i] / conf_mat_train[i, :].sum() if conf_mat_train[i, :].sum() > 0 else 0
                for i in range(7)
            ])
            recall_test = np.array([
                conf_mat_test[i, i] / conf_mat_test[i, :].sum() if conf_mat_test[i, :].sum() > 0 else 0
                for i in range(7)
            ])

            precision_train = np.array([
                conf_mat_train[i, i] / conf_mat_train[:, i].sum() if conf_mat_train[:, i].sum() > 0 else 0
                for i in range(7)
            ])
            precision_test = np.array([
                conf_mat_test[i, i] / conf_mat_test[:, i].sum() if conf_mat_test[:, i].sum() > 0 else 0
                for i in range(7)
            ])

            accuracy_fold_train.append(accuracy_train)
            accuracy_fold_test.append(accuracy_test)

            recall_fold_train.append(recall_train)
            recall_fold_test.append(recall_test)

            precision_fold_train.append(precision_train)
            precision_fold_test.append(precision_test)

        # Compute mean across folds
        accuracy_train = np.mean(accuracy_fold_train)
        accuracy_test = np.mean(accuracy_fold_test)

        recall_train = np.mean(recall_fold_train, axis=0)
        recall_test = np.mean(recall_fold_test, axis=0)

        precision_train = np.mean(precision_fold_train, axis=0)
        precision_test = np.mean(precision_fold_test, axis=0)

        average_recall_train = np.mean(recall_train)
        average_recall_test = np.mean(recall_test)

        average_precision_train = np.mean(precision_train)
        average_precision_test = np.mean(precision_test)

        f1_train_average = 2 * (average_precision_train * average_recall_train) / (average_precision_train + average_recall_train)
        f1_test_average = 2 * (average_precision_test * average_recall_test) / (average_precision_test + average_recall_test) 

        row = row.drop("results")
        row_dict = row.to_dict()

        metrics_cnn.append({
            **row_dict,
            "iteration": i,
            "accuracy_train": accuracy_train,
            "accuracy_test": accuracy_test,
            "recall_train": recall_train.tolist(),  # Convert to list for DataFrame compatibility
            "recall_test": recall_test.tolist(),
            "averaged_recall_train": average_recall_train,
            "averaged_recall_test": average_recall_test,
            "precision_train": precision_train.tolist(),
            "precision_test": precision_test.tolist(),
            "averaged_precision_train": average_precision_train,
            "averaged_precision_test": average_precision_test,
            "f1_train_average": f1_train_average,
            "f1_test_average": f1_test_average
        })

    metrics_cnn = pd.DataFrame(metrics_cnn)
    return metrics_cnn


In [28]:
data_cnn = pd.read_json('results_cnn_1.json').T

In [29]:
metrics_cnn = get_metrics(data_cnn)

In [23]:
# get 50 best performances in terms of test accuracy
best_cnn = metrics_cnn.sort_values(by="accuracy_test", ascending=False).head(50)
data_cnn_50_best = data_cnn.loc[best_cnn.index]
data_cnn_50_best.to_csv("data_cnn_50_best.csv")

## Impact of the different Hyper-parameters

In [48]:
metrics_dnn["model"] = "DNN"
metrics_cnn["model"] = "CNN"

metrics_all = pd.concat([metrics_cnn, metrics_dnn], ignore_index=True)
metrics_all["learning_rate"] = metrics_all["learning_rate"].astype(str)
metrics_all["batch_size"] = metrics_all["batch_size"].astype(str)

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define hyperparameters and metrics
hyperparameters = ['activation_function', 'n_layers', 'batch_size', 
                'learning_rate', 'loss_function', 'optimizer',
                'pooling', 'n_conv_layers']

metrics = ['accuracy_train', 'accuracy_test']

metrics_to_str = {
    'accuracy_train': 'Accuracy (Train)',
    'accuracy_test': 'Accuracy (Validation)'
}

# Grid layout settings
params_per_row = 2  # Two parameters per row
n_rows = -(-len(hyperparameters) // params_per_row)  # Ceiling division for rows

# Create subplots
fig = make_subplots(
    rows=n_rows, cols=params_per_row * 2,  # Each parameter has 2 charts (train & test)
    subplot_titles=[f"{param} - {metrics_to_str[metric]}" for param in hyperparameters for metric in metrics],
    shared_yaxes=False,
    horizontal_spacing=0.05, vertical_spacing=0.05
)

# Loop through each hyperparameter
for i, param in enumerate(hyperparameters):
    row = i // params_per_row + 1  # Compute row index

    df_grouped = metrics_all.groupby([param, "model"])[metrics].mean().reset_index()

    # Add plots for accuracy_train and accuracy_test
    for j, metric in enumerate(metrics):

        col = i % params_per_row * 2 + j + 1  # Compute column index

        for model, color in zip(["DNN", "CNN"], ["#636EFA", "#EF553B"]):

            df_subset = df_grouped[df_grouped["model"] == model]
            
            fig.add_trace(
                go.Bar(
                    x=df_subset[param], 
                    y=df_subset[metric], 
                    name=f"{model}",
                    marker_color=color,
                    legendgroup=model,
                    showlegend=(i == 0 and j == 0)  # Show legend only once
                ),
                row=row, col=col
            )

# Update layout
fig.update_layout(
    title_text="Impact of Hyperparameters on Accuracy (CNN vs. DNN)",
    height=300 * n_rows, width=1400,
    showlegend=True
)

# Show figure
#fig.show()

a = 1


## Overall Performance

In [82]:
# get the configuration with the best test accuracy

best_cnn = metrics_cnn.sort_values(by="accuracy_test", ascending=False).head(1)
best_dnn = metrics_dnn.sort_values(by="accuracy_test", ascending=False).head(1)

In [77]:
# bar plot with train and test accuracy

fig = go.Figure()

fig.add_trace(go.Bar(
    x=["CNN", "DNN"],
    y=[best_cnn["accuracy_train"].values[0], best_dnn["accuracy_train"].values[0]],
    name='Train',
    marker_color='#636EFA'
))

fig.add_trace(go.Bar(
    x=["CNN", "DNN"],
    y=[best_cnn["accuracy_test"].values[0], best_dnn["accuracy_test"].values[0]],
    name='Validation',
    marker_color='#EF553B'
))

fig.update_layout(
    title='Train and Test Accuracy for the Best CNN and DNN Models',
    barmode='group',
    xaxis_title='Model',
    yaxis_title='Accuracy',
    showlegend=True
)

fig.show()

In [83]:
# bar plot with train and test accuracy

fig = go.Figure()

fig.add_trace(go.Bar(
    x=["CNN", "DNN"],
    y=[best_cnn["f1_train_average"].values[0], best_dnn["f1_train_average"].values[0]],
    name='Train',
    marker_color='#636EFA'
))

fig.add_trace(go.Bar(
    x=["CNN", "DNN"],
    y=[best_cnn["f1_test_average"].values[0], best_dnn["f1_test_average"].values[0]],
    name='Validation',
    marker_color='#EF553B'
))

fig.update_layout(
    title='Train and Test Accuracy for the Best CNN and DNN Models',
    barmode='group',
    xaxis_title='Model',
    yaxis_title='Accuracy',
    showlegend=True
)

fig.show()

# FIXING ATTEMPTS

## Early Stopping

In [16]:
data_cnn_early_stopping = pd.read_json('results_cnn_early.json').T
#data_cnn_early_stopping

In [26]:
metrics_early = get_metrics(data_cnn_early_stopping)

In [28]:
max_f1_train = metrics_early["f1_train_average"].max()
max_f1_test = metrics_early["f1_test_average"].max()

max_f1_test, max_f1_train

(np.float64(0.6309427591407378), np.float64(0.8425808460199662))

## No Data Augmentation

In [17]:
data_no_aug_cnn = pd.read_json('../results_original/results_cnn.json').T

#data_no_aug_cnn

In [22]:
import ast

results_all = []
j = 0

for i, row in data_no_aug_cnn.iterrows():

    conf_mat_train = np.array(row["confusion_matrix_train"])
    conf_mat_test = np.array(row["confusion_matrix_val"])

    total_train = conf_mat_train.sum()
    total_test = conf_mat_test.sum()

    accuracy_train = conf_mat_train.diagonal().sum() / total_train if total_train > 0 else 0
    accuracy_test = conf_mat_test.diagonal().sum() / total_test if total_test > 0 else 0

    # Compute recall and precision (per class)
    recall_train = np.array([
        conf_mat_train[i, i] / conf_mat_train[i, :].sum() if conf_mat_train[i, :].sum() > 0 else 0
        for i in range(7)
    ])
    recall_test = np.array([
        conf_mat_test[i, i] / conf_mat_test[i, :].sum() if conf_mat_test[i, :].sum() > 0 else 0
        for i in range(7)
    ])

    precision_train = np.array([
        conf_mat_train[i, i] / conf_mat_train[:, i].sum() if conf_mat_train[:, i].sum() > 0 else 0
        for i in range(7)
    ])
    precision_test = np.array([
        conf_mat_test[i, i] / conf_mat_test[:, i].sum() if conf_mat_test[:, i].sum() > 0 else 0
        for i in range(7)
    ])

    average_recall_train = np.mean(recall_train)
    average_precision_train = np.mean(precision_train)

    average_recall_test = np.mean(recall_test)
    average_precision_test = np.mean(precision_test)

    f1_score_train = 2 * (average_precision_train * average_recall_train) / (average_precision_train + average_recall_train)
    f1_score_test  = 2 * (average_precision_test  * average_recall_test)  / (average_precision_test  + average_recall_test)

    results_all.append({
        "iteration": j,
        "accuracy_train": accuracy_train,
        "accuracy_test": accuracy_test,
        "recall_train": recall_train.tolist(),
        "recall_test": recall_test.tolist(),
        "averaged_recall_train": average_precision_train,
        "averaged_recall_test": average_recall_test,
        "precision_train": precision_train,
        "precision_test": precision_test,
        "averaged_precision_train": average_precision_train,
        "averaged_precision_test": average_precision_test,
        "f1_score_train": f1_score_train,
        "f1_score_test": f1_score_test
    })
    
    j += 1

metrics_cnn_no_aug = pd.DataFrame(results_all)
#metrics_cnn_no_aug

## Only undersampling

In [7]:
data_under_cnn = pd.read_json('./results_cnn_under.json').T

In [21]:
metrics_under_cnn = get_metrics(data_under_cnn)

In [24]:
max_f1_train = metrics_under_cnn["f1_train_average"].max()
max_f1_test = metrics_under_cnn["f1_test_average"].max()

max_f1_test, max_f1_train

(np.float64(0.4991124052880161), np.float64(0.8951487107489473))

In [25]:
metrics_under_cnn

,n_epochs,activation_function,pooling,n_conv_layers,conv_out_channels,conv_kernel_size,conv_padding,n_layers,batch_size,learning_rate,...,recall_train,recall_test,averaged_recall_train,averaged_recall_test,precision_train,precision_test,averaged_precision_train,averaged_precision_test,f1_train_average,f1_test_average
0,50,relu,MaxPool,2,16,3,1,3,64,0.100,...,"[0.5708145074362493, 0.6722284477331033, 0.700...","[0.2570538163807547, 0.4941118667917449, 0.519...",0.640481,0.423470,"[0.6487496570260732, 0.6525686713144413, 0.728...","[0.338262599469496, 0.4652747643504854, 0.5134...",0.664621,0.411867,0.652327,0.417588
1,50,relu,MaxPool,2,16,3,1,3,32,0.100,...,"[0.2667036979668537, 0.59770781696479, 0.61002...","[0.16741115828835823, 0.516666353971232, 0.509...",0.510565,0.424740,"[0.60316803048649, 0.5788547136527008, 0.62885...","[0.3967332358044433, 0.5077873906562503, 0.535...",0.537513,0.441306,0.523693,0.432865
2,50,relu,AvgPool,2,16,3,1,3,32,0.001,...,"[0.8840297892359498, 0.7912828970081431, 0.791...","[0.3645774175879614, 0.4733657754846779, 0.443...",0.817665,0.445522,"[0.6794599904920154, 0.8371888719636378, 0.900...","[0.2723646723646723, 0.4965478846139105, 0.519...",0.835924,0.460676,0.826694,0.452972
3,50,relu,AvgPool,2,16,3,1,3,64,0.001,...,"[0.7489530581259094, 0.7549545852269095, 0.779...","[0.2713002439397715, 0.47063899312070046, 0.46...",0.779778,0.433170,"[0.756708221527661, 0.8043124359356113, 0.8207...","[0.3081515151515152, 0.5157683723917332, 0.489...",0.815004,0.455181,0.797002,0.443903
4,50,relu,AvgPool,2,16,3,1,3,32,0.001,...,"[0.8778583378575616, 0.7748224073363719, 0.839...","[0.2920378099781937, 0.39994895247029394, 0.53...",0.839087,0.425725,"[0.7973990326914575, 0.8742517190506254, 0.852...","[0.3013717239596182, 0.471537895989515, 0.5000...",0.858667,0.459272,0.848764,0.441863
5,50,relu,MaxPool,2,16,3,1,3,32,0.100,...,"[0.5055435972441975, 0.5797845819667875, 0.703...","[0.19968375000085797, 0.4459131488430269, 0.50...",0.624848,0.434031,"[0.6271560281992667, 0.675939064366698, 0.7406...","[0.2538946475116688, 0.5780809766450913, 0.523...",0.672696,0.427897,0.647889,0.430942
6,50,relu,MaxPool,2,16,3,1,3,32,0.001,...,"[0.7885345843216978, 0.8369334825376417, 0.753...","[0.3446818092588665, 0.46774734208880553, 0.42...",0.783463,0.424350,"[0.7543510713765418, 0.8210543603984559, 0.853...","[0.3132411750868308, 0.470925249865342, 0.4953...",0.855350,0.489643,0.817830,0.454664
7,50,relu,AvgPool,2,16,3,1,3,64,0.001,...,"[0.7258968340815948, 0.8091551131172967, 0.830...","[0.20313271964017168, 0.5238135553470918, 0.46...",0.792901,0.415738,"[0.8701022720398868, 0.8101061081094187, 0.847...","[0.3399095408772828, 0.4867223780191221, 0.499...",0.843545,0.456906,0.817439,0.435351
8,50,relu,AvgPool,1,16,3,1,3,32,0.001,...,"[0.7886452278464151, 0.7705187068251009, 0.832...","[0.2931055097853893, 0.4918040963101939, 0.530...",0.810321,0.429834,"[0.7561053945384242, 0.8428485490648185, 0.825...","[0.30391348186697026, 0.5459832234228942, 0.50...",0.834669,0.438648,0.822315,0.434196
9,50,relu,AvgPool,2,16,3,1,3,64,0.001,...,"[0.8948298741121811, 0.886841257128512, 0.8806...","[0.30792815410898494, 0.46256793308317706, 0.5...",0.893197,0.439225,"[0.882400318643571, 0.8789470808017198, 0.9161...","[0.3432608695652174, 0.49983327884929707, 0.51...",0.897109,0.437982,0.895149,0.438603
